## Tractography-Informed nTMS Language Mapping

The purpose of this notebook is to outline the methods used in performing tractography-informed navigated transcranial magnetic stimulation (nTMS) lanugage mapping by visualizing the cortical end points of language-relevant white matter tracts (i.e. the Arcuate Fasciculus). These endpoints can then be used as stimulation markers for nTMS language mapping.

### Software Requirements

[**MRtrix**](https://www.mrtrix.org/download/): version 3.0

[**FSL**](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation): parts of the MRtrix3 script require FSL tools

### Optional Software
[**Karawun**](https://github.com/DevelopmentalImagingMCRI/karawun): python package that converts diffusion weighted imaging (DWI) and tractography files into forms that are readable in [**BrainLab**](https://www.brainlab.com/).

## Data Structure and Overview

**Required**: DWI and T1 files.

Preprocessing performed using MRtrix3 and FSL through the following steps: denoising, removal of Gibbs-ringing artifacts, correction of subject motion, eddy-current correction, susceptibility-induced distortion correction, and bias field correction. A Fiber Orientation Distribution (FOD) Map was estimated and subsequent region of interest (ROI)-based Tractography was performed using MRtrix3. Probabilistic Tractography was performed using the iFOD2 algorithm (default).

## Preprocessing
Preprocessing commands with MRtrix3 are thoroughly described on the [**MRtrix website**](https://mrtrix.readthedocs.io/en/latest/index.html) and in the [**B.A.T.M.A.N. Tutorial**](https://osf.io/fkyht/)

In [ ]:
!mrconvert t1*/ t1.nii.gz
!mrconvert ep2dif*/ dwi.mif

In [ ]:
!N4BiasFieldCorrection -d 3 -i t1.nii.gz -o t1_N4.nii.gz
!5ttgen fsl t1_N4.nii.gz 5tt.mif

In [ ]:
#denoising
!dwidenoise dwi.mif dwi_den.mif -noise noise.mif

In [ ]:
#optional: inspect the results 
!mrcalc dwi.mif dwi_den.mif –subtract residual.mif

In [ ]:
#Unringing and removal of Gibb's ringing artifacts 
#The “axes” option must be adjusted to your dataset: 
#With this option, you inform the algorithm of the plane in which you acquired your data:
# –axes 0,1 means you acquired axial slices; -axes 0,2 refers to coronal slices and –axes 1,2 to sagittal slices!
!mrdegibbs dwi_den.mif dwi_den_unr.mif -axes 0,1

In [ ]:
!dwifslpreproc -rpe_none -pe_dir AP dwi_den_unr.mif dwi_den_unr_pre.mif -nocleanup -eddy_options "--slm=linear "
#if you do the b0 step, then use the following command:
#dwipreproc dwi_den_unr.mif dwi_den_unr_preproc.mif –pe_dir AP –rpe_pair –se_epi b0_pair.mif –eddy_options “ --slm=linear”

In [ ]:
#calculate the number of outlier slices
!cd dwipreproc-tmp-*
!totalSlices=`mrinfo dwi.mif | grep Dimensions | awk '{print $6 * $8}'`
!totalOutliers=`awk '{ for(i=1;i<=NF;i++)sum+=$i } END { print sum }' dwi_post_eddy.eddy_outlier_map`
!echo "If the following number is greater than 10, you may have to discard this subject because of too much motion or corrupted slices"
!echo "scale=5; ($totalOutliers / $totalSlices * 100)/1" |bc | tee percentageOutliers.txt

In [ ]:
#bias field correction
!dwibiascorrect ants dwi_den_unr_pre.mif dwi_den_unr_pre_unbia.mif -bias bias.mif

In [ ]:
#upscaling of the dmri data
!mrgrid dwi_den_unr_pre_unbia.mif regrid -vox 1.3 dwi_den_unr_pre_unbia_up.mif

In [ ]:
#mask estimation
!dwi2mask dwi_den_unr_pre_unbia.mif mask_up.mif

In [ ]:
#fiber orientation distribution
!dwi2response dhollander dwi_den_unr_pre_unbia.mif wm.txt gm.txt csf.txt
!dwi2fod msmt_csd dwi_den_unr_pre_unbia.mif csf.txt csf.mif wm.txt wm.mif -mask mask_up.mif

## Tractography

In the following steps, the delineation of the arcuate fasciculus will be explained. The seeding of the ROIs was performed using the guidelines outlined in [(Fekonja et al., 2019)](https://link.springer.com/article/10.1007/s00701-019-03899-0)


In [ ]:
#seeding the ROIs
!mrview t1.nii.gz
#using the ROI editor, set the ROIs to include / exclude and save them 

In [ ]:
#creating tractogram 
!tckgen wm.mif -seed_im ROI1.mif -incl ROI2.mif AF.tck -sel 5k -cuto 0.16 -minl 50 -excl ROI3.mif -excl ROI4.mif  
#to overwrite after excluded ROIs have been set use -force

## Mapping the Cortical Endpoints
From the tck file, the cortical ends are mapped to a mif file

In [ ]:
#display only the cortical endpoints
tckmap -ends_only -template wm.mif AF.tck AF_ends.mif
#the endpoints of the tck file are now displayed as points in a .mif file

In [ ]:
#Viewing the cortical endpoints
mrview t1.nii.gz -overlay.load AF_ends.mif
#make sure the endpoints overlay correctly on the original t1 image

##  Mapping of Endpoints onto the T1 image
The endpoints of the AF are then transformed onto the original T1 image, which can then be uploaded into the nTMS system (i.e. Nexstim)

In [ ]:
mrtransform -interp nearest -template t1.nii.gz AF_ends.mif AF_transform.mif

In [ ]:
mrcalc AF_transform.mif 1000 -multi t1.nii.gz -max AF_transf_max.mif
# the output image is scaled up a factor of 1000 (these intensities can be edited as need-be)

## Exporting the T1 Image File containing the Cortical Endpoints
The image needs to be uploaded into a readable file for the nTMS machine to read (in this case, NIfTI)

In [ ]:
mrtransform AF_trans_max.mif AF_export.nii.gz 
#NextStim requires the t1 image to be in a readable format 
#NIfTI file are readable, .mif files are not

## Transforming Exported NIfTI files into a readable Format for Brainlab 

**Optional:** This step is useful if nTMS data needs to be visualized/edited in Brainlab following the nTMS language mapping

In [ ]:
#activate environment after installation
!conda activate KarawunEnv

In [ ]:
#converting files into dicoms which are placed into output folder readable for brainlab
!importTractography --dicom-template t1/*.dcm --nifti t1.nii.gz --nifti AF_export.nii.gz --tract-files AF_left.tck AF_right.tck -o brainlab_output_folder